In [20]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import os 
from itertools import cycle
color_cycle = cycle(plt.rcParams['axes.prop_cycle'].by_key()['color'])

In [21]:
INPUT_DIR = ".\\datas\\"

def read_data():
    cal = pd.read_csv(f"{INPUT_DIR}calendar.csv")
    stv = pd.read_csv(f"{INPUT_DIR}sales_train_validation.csv")
    ste = pd.read_csv(f"{INPUT_DIR}sales_train_evaluation.csv")
    ss = pd.read_csv(f"{INPUT_DIR}sample_submission.csv")
    sellp = pd.read_csv(f"{INPUT_DIR}sell_prices.csv")
    
    return cal, stv, ste, ss, sellp

In [22]:
cal, stv, ste, ss, sellp = read_data()

In [23]:
def reduce_mem_usage(df, verbose=True):
    """
    目的：メモリサイズの削減
    df: メモリを削減したい DataFrame (pandas.DataFrame)
    verbose: 実行時に、メモリ削減の情報を出力するかどうかを指定(bool)

    ■ 基本思想
    【前提知識】
    pandas で作成したデータフレームのうち数値データは、特に dtype を指定しない場合
    int64 または float64 でデータを作成するので、
    実際のデータよりもこの型が大きいと余計なメモリサイズを確保してしまう。

    【処理内容】
    (1) 入力された DataFrame の column の型を全てチェック(for loop)
    (2) その型が大きい数値データ(int16~int64, float16~float64)ならば、
        そのデータフレームの最大値・最小値をチェック。
        現在処理中のカラムを、上記の最大値・最小値を表せる必要最低限の型に変換する。
        int と floatに分けて処理。

    ────────────────────────────────────────────────────────────────────────
    【変更履歴】
    2020/06/06:
    ■ 35行目
    ifのネストが深かったので、リファクタ。
    Early Continueを入れたので可読性が向上(したはず)。

    ■ 46行目・71行目(置き換え・追加)
    説明変数(関数?)で置き換え。
    columnのtypeがintであるか否かを判定する関数を噛ませている。
    (返り値はbool値)
    """

    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2

    # main loop    
    for col in df.columns:
        col_type = df[col].dtypes

        if col_type not in numerics: 
            continue # Early continue if column type is not numeric
        
        c_min = df[col].min()
        c_max = df[col].max()

        if IsInt(col_type):
            if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                df[col] = df[col].astype(np.int8)
            elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                df[col] = df[col].astype(np.int16)
            elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                df[col] = df[col].astype(np.int32)
            elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                df[col] = df[col].astype(np.int64)  
        else:
            if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                df[col] = df[col].astype(np.float16)
            elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                df[col] = df[col].astype(np.float32)
            else:
                df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2

    if verbose: 
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))

    return df


def IsInt(col_type):
    return str(col_type)[:3] == 'int'


In [24]:
dfs = [cal, stv, ste, ss, sellp]
for df in dfs:
    df = reduce_mem_usage(df)

Mem. usage decreased to  0.12 Mb (41.9% reduction)
Mem. usage decreased to 95.00 Mb (78.7% reduction)
Mem. usage decreased to 96.13 Mb (78.8% reduction)
Mem. usage decreased to  2.09 Mb (84.5% reduction)
Mem. usage decreased to 130.48 Mb (37.5% reduction)


In [25]:
from sklearn import preprocessing, metrics
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM,Dropout
from keras.layers import RepeatVector,TimeDistributed
from numpy import array
from keras.models import Sequential, load_model
#import utils_paths
import re
from tqdm import tqdm
import os

In [26]:
train_sales = stv
calendar = cal
sell_prices = pd.read_csv(f"{INPUT_DIR}sell_prices.csv")
# pd.pivot() を使うときに、メモリサイズを削減したものだとエラーになる模様。なので再度読み直し。対応策はないのか？
submission_file = ss

In [27]:
def transform(data):
    
    nan_features = ['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
    for feature in nan_features:
        data[feature].fillna('unknown', inplace = True)
        
    cat = ['event_name_1','event_type_1','event_name_2','event_type_2','snap_CA','snap_TX','snap_WI']
    for feature in cat:
        encoder = preprocessing.LabelEncoder()
        data[feature] = encoder.fit_transform(data[feature])
    
    return data
#calendarの何もない日をunknownで埋めて、label encoderによって処理しやすい形に変える
# one-hot encodingのほうが良いのでは？

In [28]:
days = range(1, 1970)
time_series_columns = [f'd_{i}' for i in days]

event_snap_columns = ['event_name_1','event_type_1','event_name_2','event_type_2','snap_CA','snap_TX','snap_WI']

transfer_cal = pd.DataFrame(calendar[event_snap_columns].values.T,
                            index=event_snap_columns,
                            columns=time_series_columns)
transfer_cal = transfer_cal.fillna(0)

event_name_1_se = transfer_cal.loc['event_name_1'].apply(lambda x: x if re.search("^\d+$", str(x)) else np.nan).fillna(10)
event_name_2_se = transfer_cal.loc['event_name_2'].apply(lambda x: x if re.search("^\d+$", str(x)) else np.nan).fillna(10)
#↑event nameが入っているものをnanにした後、10にしている。

In [66]:
calendar.loc[calendar["d"] == "d_1919"]

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
1918,2016-04-30,11614,Saturday,1,4,2016,d_1919,21,2,2,2,0,0,0


In [80]:
"""
※注意※

ここで、使用メモリを減らすためにcalenderの範囲が減らされている。
増やすと単純に精度向上が可能？
-> もしフルに使うと、ローカルのメモリが死ぬ

"""

calendar['date'] = pd.to_datetime(calendar['date'])
calendar = calendar[calendar['date']>= '2014-1-27']  #reduce memory
calendar = calendar[calendar["date"] <= "2016-05-22"] #eliminate evaluate date
#使うデータを少なくします。
calendar= transform(calendar)
# Attempts to convert events into time series data.
transfer_cal = pd.DataFrame(calendar[event_snap_columns + ["date", "d"]].values.T,
                            index=event_snap_columns + ["date", "d"])
transfer_cal

,0,1,2,3,4,5,6,7,8,9,...,837,838,839,840,841,842,843,844,845,846
event_name_1,30,30,30,30,30,30,26,30,30,30,...,30,30,30,30,30,30,30,30,30,30
event_type_1,4,4,4,4,4,4,3,4,4,4,...,4,4,4,4,4,4,4,4,4,4
event_name_2,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
event_type_2,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
snap_CA,0,0,0,0,0,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
snap_TX,0,0,0,0,0,1,0,1,0,1,...,1,0,1,0,0,0,0,0,0,0
snap_WI,0,0,0,0,0,0,1,1,0,1,...,0,1,1,0,0,0,0,0,0,0
date,2014-01-27 00:00:00,2014-01-28 00:00:00,2014-01-29 00:00:00,2014-01-30 00:00:00,2014-01-31 00:00:00,2014-02-01 00:00:00,2014-02-02 00:00:00,2014-02-03 00:00:00,2014-02-04 00:00:00,2014-02-05 00:00:00,...,2016-05-13 00:00:00,2016-05-14 00:00:00,2016-05-15 00:00:00,2016-05-16 00:00:00,2016-05-17 00:00:00,2016-05-18 00:00:00,2016-05-19 00:00:00,2016-05-20 00:00:00,2016-05-21 00:00:00,2016-05-22 00:00:00
d,d_1095,d_1096,d_1097,d_1098,d_1099,d_1100,d_1101,d_1102,d_1103,d_1104,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941


In [62]:
price_fea = calendar[['wm_yr_wk','date']].merge(sell_prices, on = ['wm_yr_wk'], how = 'left')
#販売価格に対して、calendarを結合した。
price_fea['id'] = price_fea['item_id']+'_'+price_fea['store_id']+'_validation'
df = price_fea.pivot('id','date','sell_price')
#各商品を一行にして、商品の値段の移り変わりを表した。

In [63]:
price_df = train_sales.merge(df, on=['id'], how= 'left').iloc[:,-len(df.columns):]
price_df.index = train_sales.id
price_df.head()
#train salesに値段の移り変わりをのデータをjoinした。

,2014-01-27,2014-01-28,2014-01-29,2014-01-30,2014-01-31,2014-02-01,2014-02-02,2014-02-03,2014-02-04,2014-02-05,...,2016-05-13,2016-05-14,2016-05-15,2016-05-16,2016-05-17,2016-05-18,2016-05-19,2016-05-20,2016-05-21,2016-05-22
id,,,,,,,,,,,,,,,,,,,,,
HOBBIES_1_001_CA_1_validation,8.26,8.26,8.26,8.26,8.26,8.26,8.26,8.26,8.26,8.26,...,8.38,8.38,8.38,8.38,8.38,8.38,8.38,8.38,8.38,8.38
HOBBIES_1_002_CA_1_validation,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,...,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97
HOBBIES_1_003_CA_1_validation,NaN,NaN,NaN,NaN,NaN,2.97,2.97,2.97,2.97,2.97,...,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97
HOBBIES_1_004_CA_1_validation,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,...,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64
HOBBIES_1_005_CA_1_validation,3.08,3.08,3.08,3.08,3.08,3.08,3.08,3.08,3.08,3.08,...,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88


In [64]:
price_df = reduce_mem_usage(price_df)

Mem. usage decreased to 49.49 Mb (74.9% reduction)


In [65]:
df = reduce_mem_usage(df)

Mem. usage decreased to 49.49 Mb (74.9% reduction)


In [34]:
def min_max(df):
    return (df-df.mean())/df.std()  #scale

In [35]:
def Normalize(list):
    list = np.array(list)
    #配列の状態から行列に変換
    low, high = np.percentile(list, [0, 100])#todo ここの細かい動きが追えていない
    delta = high - low
    if delta != 0:
        for i in range(0, len(list)):
            list[i] = (list[i]-low)/delta
    return  list,low,high
#[0 1]に変換

def FNoramlize(list,low,high):
    delta = high - low
    if delta != 0:
        for i in range(0, len(list)):
            list[i] = list[i]*delta + low
    return list
#normalizeされた値を元に戻す

def Normalize2(list,low,high):
    list = np.array(list)
    delta = high - low
    if delta != 0:
        for i in range(0, len(list)):
            list[i] = (list[i]-low)/delta
    return  list

以下訓練データ生成

In [59]:
calendar.loc[calendar["d"] == "d_1941"]

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
1940,2016-05-22,11617,Sunday,2,5,2016,d_1941,30,4,2,2,0,0,0


In [39]:
#price_df.isnull().sum()

2014-01-27    4167
2014-01-28    4167
2014-01-29    4167
2014-01-30    4167
2014-01-31    4167
              ... 
2016-06-15       0
2016-06-16       0
2016-06-17       0
2016-06-18       0
2016-06-19       0
Length: 875, dtype: int64

In [40]:
#price_df.fillna(0)

,2014-01-27,2014-01-28,2014-01-29,2014-01-30,2014-01-31,2014-02-01,2014-02-02,2014-02-03,2014-02-04,2014-02-05,...,2016-06-10,2016-06-11,2016-06-12,2016-06-13,2016-06-14,2016-06-15,2016-06-16,2016-06-17,2016-06-18,2016-06-19
id,,,,,,,,,,,,,,,,,,,,,
HOBBIES_1_001_CA_1_validation,8.257812,8.257812,8.257812,8.257812,8.257812,8.257812,8.257812,8.257812,8.257812,8.257812,...,8.382812,8.382812,8.382812,8.382812,8.382812,8.382812,8.382812,8.382812,8.382812,8.382812
HOBBIES_1_002_CA_1_validation,3.970703,3.970703,3.970703,3.970703,3.970703,3.970703,3.970703,3.970703,3.970703,3.970703,...,3.970703,3.970703,3.970703,3.970703,3.970703,3.970703,3.970703,3.970703,3.970703,3.970703
HOBBIES_1_003_CA_1_validation,0.000000,0.000000,0.000000,0.000000,0.000000,2.970703,2.970703,2.970703,2.970703,2.970703,...,2.970703,2.970703,2.970703,2.970703,2.970703,2.970703,2.970703,2.970703,2.970703,2.970703
HOBBIES_1_004_CA_1_validation,4.640625,4.640625,4.640625,4.640625,4.640625,4.640625,4.640625,4.640625,4.640625,4.640625,...,4.640625,4.640625,4.640625,4.640625,4.640625,4.640625,4.640625,4.640625,4.640625,4.640625
HOBBIES_1_005_CA_1_validation,3.080078,3.080078,3.080078,3.080078,3.080078,3.080078,3.080078,3.080078,3.080078,3.080078,...,2.880859,2.880859,2.880859,2.880859,2.880859,2.880859,2.880859,2.880859,2.880859,2.880859
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FOODS_3_823_WI_3_validation,2.500000,2.500000,2.500000,2.500000,2.500000,2.880859,2.880859,2.880859,2.880859,2.880859,...,2.980469,2.980469,2.980469,2.980469,2.980469,2.980469,2.980469,2.980469,2.980469,2.980469
FOODS_3_824_WI_3_validation,2.679688,2.679688,2.679688,2.679688,2.679688,2.679688,2.679688,2.679688,2.679688,2.679688,...,2.480469,2.480469,2.480469,2.480469,2.480469,2.480469,2.480469,2.480469,2.480469,2.480469
FOODS_3_825_WI_3_validation,3.980469,3.980469,3.980469,3.980469,3.980469,3.980469,3.980469,3.980469,3.980469,3.980469,...,3.980469,3.980469,3.980469,3.980469,3.980469,3.980469,3.980469,3.980469,3.980469,3.980469


In [41]:
stv.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


In [67]:
# ste における、追加データ(d_1920~d_1941 は、評価用に残しておく(validation set. K-Foldするならまとめてもいいかも。))
train_sales = stv
train_sales["d"] = pd.Series(np.arange(1, 1920))

In [54]:
train_sales["d"].head()

0    1.0
1    2.0
2    3.0
3    4.0
4    5.0
Name: d, dtype: float64

In [68]:
train_sales = stv.set_index("id", drop=True).drop(columns=["item_id", "dept_id", "cat_id", "store_id", "state_id"]).transpose()

In [71]:
train_sales = train_sales.reset_index(drop=True)

In [76]:
train_sales = train_sales.drop(columns=["index"])

In [78]:
train_sales.shape

(1914, 30490)

In [88]:
train_calendar = transfer_cal.transpose().drop(columns=["date", "d"])

In [89]:
train_calendar

,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,30,4,2,2,0,0,0
1,30,4,2,2,0,0,0
2,30,4,2,2,0,0,0
3,30,4,2,2,0,0,0
4,30,4,2,2,0,0,0
...,...,...,...,...,...,...,...
842,30,4,2,2,0,0,0
843,30,4,2,2,0,0,0
844,30,4,2,2,0,0,0
845,30,4,2,2,0,0,0


In [91]:
train_sales_modified = train_sales[-847:].reset_index(drop=True)

In [56]:
ste

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,1,0,3,0,1,1,0,0,1,1
30486,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
30487,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,0,0,1,2,0,1,0,1,0,2
30488,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,1,1,1,4,6,0,1,1,1,0


In [121]:
def train_data_generator(feature, cal):
    """
    6/7 未完成
    for文の中にもう一つfor文を作成し、
    所望の長さ(シーケンス)のデータをジェネレートするようにしたい
    
    -> 別のセルで、keras用のgeneratorクラスを作成することにした。
    これ自体は、トレーニングデータ作成に使えるので残す。

    feature: データ点列 × データ数のデータフレーム
    cal: カレンダーのデータフレーム

    引数は特徴量を追加した段階で足すかもしれない。

    """

    length = feature.shape[1]
    columns = feature.columns
    for i in range(0, length):
        feature_df =  pd.concat([cal.reset_index(drop=True), feature[columns[i]]], axis=1)
        yield feature_df

In [122]:
# Train sales modified に price のデータが付加されていない。
tdg = train_data_generator(train_sales_modified, train_calendar)

In [123]:
OUTPUT_PATH = ".\\datas\\training_datas\\"
for i, df in enumerate(tdg):
    df.to_csv(OUTPUT_PATH + "train_data" + str(i) + ".csv", index=False)

ここまででデータ作成しているので、以下ジェネレータでファイルを読み込むところからスタート可<br />
※ price のデータを結合し忘れた。やり直し。

In [124]:
NUM_ITEMS = 30490
OUTPUT_PATH = ".\\datas\\training_datas\\"

def train_data_from_csv_generator(num=NUM_ITEMS):
    for i in range(NUM_ITEMS):
        df = pd.read_csv(OUTPUT_PATH + "train_data" + str(i) +".csv")
        yield df



In [125]:
train_generator = train_data_from_csv_generator(num=10) 

In [126]:
next(train_generator)

,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,HOBBIES_1_001_CA_1_validation
0,30,4,2,2,0,0,0,0.0
1,30,4,2,2,0,0,0,1.0
2,30,4,2,2,0,0,0,1.0
3,30,4,2,2,0,0,0,0.0
4,30,4,2,2,0,0,0,0.0
...,...,...,...,...,...,...,...,...
842,30,4,2,2,0,0,0,3.0
843,30,4,2,2,0,0,0,0.0
844,30,4,2,2,0,0,0,1.0
845,30,4,2,2,0,0,0,1.0
